In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

<h2> Hotel Reviews </h2>

In [2]:
hotel_reviews=pd.read_csv('hotels/hotel_reviews.csv')
hotel_reviews.head(3)

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN


In [3]:
hotel_reviews=hotel_reviews[['reviews.text','reviews.rating']]
hotel_reviews

,reviews.text,reviews.rating
0,Pleasant 10 min walk along the sea front to th...,4.0
1,Really lovely hotel. Stayed on the very top fl...,5.0
2,Ett mycket bra hotell. Det som drog ner betyge...,5.0
3,We stayed here for four nights in October. The...,5.0
4,We stayed here for four nights in October. The...,5.0
...,...,...
35907,Best of the Best!!!! My family and I stayed in...,5.0
35908,Amazing Vacation in Beautiful Cabin We stayed ...,5.0
35909,My husband and I stayed at Tree Tops which is ...,5.0
35910,to share your opinion of this businesswith YP ...,0.0


In [4]:
hotel_reviews.dropna(inplace=True)
hotel_reviews

,reviews.text,reviews.rating
0,Pleasant 10 min walk along the sea front to th...,4.0
1,Really lovely hotel. Stayed on the very top fl...,5.0
2,Ett mycket bra hotell. Det som drog ner betyge...,5.0
3,We stayed here for four nights in October. The...,5.0
4,We stayed here for four nights in October. The...,5.0
...,...,...
35907,Best of the Best!!!! My family and I stayed in...,5.0
35908,Amazing Vacation in Beautiful Cabin We stayed ...,5.0
35909,My husband and I stayed at Tree Tops which is ...,5.0
35910,to share your opinion of this businesswith YP ...,0.0


In [5]:
hotel_reviews = hotel_reviews[(hotel_reviews['reviews.rating']>0) & (hotel_reviews['reviews.rating']<=5)]
hotel_reviews

,reviews.text,reviews.rating
0,Pleasant 10 min walk along the sea front to th...,4.0
1,Really lovely hotel. Stayed on the very top fl...,5.0
2,Ett mycket bra hotell. Det som drog ner betyge...,5.0
3,We stayed here for four nights in October. The...,5.0
4,We stayed here for four nights in October. The...,5.0
...,...,...
35905,We truly enjoyed staying at the Elk Springs re...,5.0
35906,We were here for a 2nd time the last weekend o...,5.0
35907,Best of the Best!!!! My family and I stayed in...,5.0
35908,Amazing Vacation in Beautiful Cabin We stayed ...,5.0


In [6]:
avg_mask=hotel_reviews['reviews.rating']==3
pos_mask=hotel_reviews['reviews.rating']>3
neg_mask=hotel_reviews['reviews.rating']<3

In [7]:
hotel_reviews.loc[ avg_mask, 'reviews.rating'] ='avg'
hotel_reviews.loc[ pos_mask, 'reviews.rating'] ='pos'
hotel_reviews.loc[ neg_mask, 'reviews.rating'] ='neg'

In [8]:
hotel_reviews

,reviews.text,reviews.rating
0,Pleasant 10 min walk along the sea front to th...,pos
1,Really lovely hotel. Stayed on the very top fl...,pos
2,Ett mycket bra hotell. Det som drog ner betyge...,pos
3,We stayed here for four nights in October. The...,pos
4,We stayed here for four nights in October. The...,pos
...,...,...
35905,We truly enjoyed staying at the Elk Springs re...,pos
35906,We were here for a 2nd time the last weekend o...,pos
35907,Best of the Best!!!! My family and I stayed in...,pos
35908,Amazing Vacation in Beautiful Cabin We stayed ...,pos


In [9]:
train_data, test_data = train_test_split(hotel_reviews, test_size=0.3)

In [10]:
train_text=train_data['reviews.text']
test_text=test_data['reviews.text']
train_labels=train_data['reviews.rating']
test_labels=test_data['reviews.rating']

In [11]:

#Build a counter based on the training dataset
counter = CountVectorizer()
counter.fit(train_text)


#count the number of times each term appears in a document and transform each doc into a count vector
counts_train = counter.transform(train_text)#transform the training data
counts_test = counter.transform(test_text)#transform the testing data

vocabulary=[x[0] for x in sorted(counter.vocabulary_.items(),key=lambda x:x[1])]


In [32]:
model1 = DecisionTreeClassifier()
model2 = KNeighborsClassifier()
model3= RandomForestClassifier()

predictors=[('dt',model1),('knn',model2),('rf',model3)]

VT=VotingClassifier(predictors)

#train all classifier on the same dataset
VT.fit(counts_train,train_labels)

#use hard voting to predict (majority voting)
pred=VT.predict(counts_test)

accuracy_score(test_labels,pred)

0.7139112707504126

<h2> Medical Diagnosis </h2>

In [33]:
med=pd.read_csv('medical/medical_data.csv')


In [34]:
train_data, test_data = train_test_split(med, test_size=0.3)

train_symptoms=train_data.drop(columns=['diagnosis'])
train_labels=train_data['diagnosis']               
    
test_symptoms=test_data.drop(columns=['diagnosis'])
test_labels=test_data['diagnosis']               
    


In [35]:
train_symptoms

,fever,cough,tiredness,headache,stuffy nose,runny nose,sneezing,sore throat
1298,0,1,0,0,0,1,1,0
740,1,0,1,0,1,1,0,1
524,1,1,1,1,1,0,0,1
135,1,0,1,0,0,0,0,0
136,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1063,0,0,0,0,1,0,1,0
846,1,1,1,1,0,1,0,1
1367,0,0,0,0,1,0,1,0
1617,0,0,0,0,1,1,1,0


In [36]:
model1 = DecisionTreeClassifier()
model2 = KNeighborsClassifier()
model3= RandomForestClassifier()

predictors=[('dt',model1),('knn',model2),('rf',model3)]

VT=VotingClassifier(predictors)

#train all classifier on the same dataset
VT.fit(train_symptoms,train_labels)

#use hard voting to predict (majority voting)
pred=VT.predict(test_symptoms)

accuracy_score(test_labels,pred)

0.8416666666666667